<a href="https://colab.research.google.com/github/JuanmaVL/SyS/blob/main/Corte1/Parcial1/parcial1_sys_2024_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parcial 1: Señales y Sistemas 2024-II

 ## Profesor: Andrés Marino Álvarez Meza, Ph.D.


## Departamento de Ingeniería Eléctrica, Electrónica, y Computación
## Universidad Nacional de Colombia - sede Manizales

# Instrucciones

-- Para recibir el crédito total, sus respuestas deben estar justificadas de manera clara, detallada y concreta, mostrando los procedimientos y razonamientos paso a paso.

-- Está permitido el uso de herramientas de inteligencia artificial (IA). Si las utiliza, por favor declare explícitamente cómo fueron empleadas en la resolución de cada pregunta. Incluya los prompts (consultas) y las iteraciones realizadas con las IA durante el desarrollo del parcial.

-- La entrega del parcial debe realizarse antes de las 23:59 del 5 de diciembre de 2024 al correo electrónico amalvarezme@unal.edu.co mediante un enlace de GitHub.

-- Los códigos deben estar debidamente comentados en las celdas correspondientes y explicados en celdas de texto (markdown). Los códigos que no incluyan comentarios ni discusiones no serán considerados en la evaluación final.

# Pregunta 1 (valor 2.5 puntos)

Cuál es la señal obtenida en tiempo discreto al utilizar un conversor análogo digital de 5 bits con frecuencia de muestreo de $5kHz$, entrada análoga de -3.3 a 3.3 [v], aplicado a la señal continua $x(t) = 0.3 \cos(1000\pi t-\pi/4) +
0.6 \sin(2000\pi t) + 0.1 \cos(11000\pi t-\pi)$?. Realizar la simulación del proceso de digitalización incluyendo al menos 3 ciclos de la señal $x(t)$.

En caso de que la digitalización no sea apropiada, diseñe e implemente un conversor adecuado para la señal estudiada. El convesor debe permitir configurar la cantidad de bits, rango de la entrada análoga y la frecuencia de muestreo, indicándole al usuario si dicha frecuencia es apropiada o no, y graficar la señal continua, discreta y digital.

In [ ]:
import numpy as np #Arreglos
import matplotlib.pyplot as plt #Libreria
from scipy.spatial.distance import cdist #Función distancia

In [ ]:
A1 = 0.3 #Amplitud 1
A2 = 0.6 #Amplitud 2
A3 = 0.1 #Amplitud 3
PH1 = np.pi/4 #Fase 1
PH3 = np.pi #Fase 3

w1 = 1000*np.pi #velocidad angular 1
w2 = 2000*np.pi #velocidad angular 2
w3 = 11000*np.pi #velocidad angular 3

T1 = 2*np.pi/w1 #Periodo 1
T2 = 2*np.pi/w2 #Periodo 2
T3 = 2*np.pi/w3 #Periodo 3

F1 = w1/(2*np.pi) #Frecuencia 1
F2 = w2/(2*np.pi) #Frecuencia 2
F3 = w3/(2*np.pi) #Frecuencia 3

FS = 2*max(F1,F2,F3) #Se define la frecuencia de muestreo nyquist
T = 1/500 #Periodo fundamental
Ts = 1/FS #Periodo de muestreo

print("Las frecuencias son:", F1,F2,F3)
print("Los periodos son:", T1,T2,T3)
print("La frecuencia de muestreo es:", FS)
print("El periodo fundamental es:", T)

In [ ]:
#Definir el vector tiempo
ti = 0
tf = 3*T

tv = np.arange(ti,tf,Ts)

In [ ]:
#Se construye la señal
X1 = A1*np.cos(w1*tv-PH1)
X2 = A2*np.sin(w2*tv)
X3 = A3*np.cos(w3*tv-PH3)
X = X1 + X2 + X3
plt.figure(figsize=(20, 10))
plt.plot(tv,X,'r',label='$X(t) = 0.3\cos(1000π*t-π/4)- 0.6 \sin(2000πt)+ 0.1\cos(11000πt-π)$')
plt.plot(tv,X1,'b',label='$x_1(t) = 0.3\cos(1000πt-π/4)$')
plt.plot(tv,X2,'g',label='$x_2(t) = 0.6\sin(2000πt)$')
plt.plot(tv,X3,'y',label='$x_3(t) = 0.1\cos(11000πt-π)$')
plt.xlabel('Tiempo (s)')
plt.ylabel('Amplitud')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
#Digitalizar la señal

y_min = 0 # valor min después de cero y pendiente
y_max = 10 # valor max después de cero y pendiente
#crear funcion en python
def my_ceropen(X,ymin,ymax): #en general se pueden definir valores por defecto
  '''
  Codigo base para simular proceso de cero y pendiente
  Se ingresa arreglo de numpy y los valores min  y max despues de cero y pendiente
  '''
  xmax = max(X) # x.max()
  xmin = min(X) # x.min()
  m = (ymax - ymin)/(xmax - xmin)
  c = ymin - m*xmin
  yv = m*X + c
  return yv

In [ ]:
ymin = -3.3
ymax = 3.3
xcp = my_ceropen(X,ymin,ymax)
plt.plot(tv,xcp,c='r',label='cero_pendiente')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
nbits = 5
num_est = 2**nbits
vq = np.linspace(ymin,ymax,num_est) # vector de cauntizacion

In [ ]:
#función cuantización punto a punto

def my_cuantizador(yn, vq) : #yn punto a #cuantizar, vq vector de estados

  Ne = vq.shape[0] #tamaño vector de estados
  dn = cdist(yn.reshape(-1,1),vq.reshape(-1,1))#distancia yn a vector estados, reshape(-1,1) asegura vectores columna para poder utilizar cdist
  ind = np.argmin(dn) #posición distancia min
  return vq[ind]

In [ ]:
for i in range(xcp.shape[0]):
   if i==0: #primera muestra
     yq = my_cuantizador(xcp[i],vq)
   else:
     yq = np.append(yq,my_cuantizador(xcp[i],vq)) #apilar datos cuantizando si i>0

In [ ]:
plt.plot(tv,xcp,c='r', label='xcp')#cs continua

plt.stem(tv,xcp, label='xcp_dis')#cs discreta

#estados de cuantización

for v in vq:
  plt.plot(tv,v*np.ones(len(tv)))

# graficar señal Digital (discreta + cuantizada)

plt.plot(tv,yq,c='y',marker='o',label='ydig', linewidth=2)

plt.legend()
plt.grid()
plt.xlabel('t')
plt.ylabel('Amplitud')
plt.show()

In [ ]:
#Se realiza el codigo para la señal interactiva


A1 = 0.3 #Amplitud 1
A2 = 0.6 #Amplitud 2
A3 = 0.1 #Amplitud 3
PH1 = np.pi/4 #Fase 1
PH3 = np.pi #Fase 3

w1 = 1000*np.pi #velocidad angular 1
w2 = 2000*np.pi #velocidad angular 2
w3 = 11000*np.pi #velocidad angular 3

T1 = 2*np.pi/w1 #Periodo 1
T2 = 2*np.pi/w2 #Periodo 2
T3 = 2*np.pi/w3 #Periodo 3

F1 = w1/(2*np.pi) #Frecuencia 1
F2 = w2/(2*np.pi) #Frecuencia 2
F3 = w3/(2*np.pi) #Frecuencia 3

T = 1/500 #Periodo fundamental
ti = 0
tf = 3*T

#Se define la frecuencia de muestres utilizando Nuquist
Fs = int(input("Ingrese la frecuencia de muestreo mayor o igual a 11000: "))

while Fs < 11000:
  print(f"La frecuencia de muestreo de {Fs}[HZ] debe ser mayor o igual a 11000 para cumplir Nyquist")
  Fs = int(input("Ingrese la frecuencia de muestreo mayor o igual a 11000: "))

else:

  tv = np.arange (ti,tf,1/Fs)

  #Se construye la señal
  x = A1*np.cos((w1*tv)-PH1) + A2*np.sin((w2*tv)) + A3*np.cos((w3*tv)-PH3)

  plt.plot(tv,x,color='b',label='$x(t)')
  plt.grid(True)#Cuadriculas
  plt.legend()
  plt.xlabel('t')
  plt.ylabel('$x(t)')
  plt.show()


In [ ]:
def my_ceropen(x,ymin,ymax): #en general se pueden definir valores por defecto
  '''
  Codigo base para simular proceso de cero y pendiente
  Se ingresa arreglo de numpy y los valores min  y max despues de cero y pendiente
  '''
  xmax = max(x) # x.max()
  xmin = min(x) # x.min()
  m = (ymax - ymin)/(xmax - xmin)
  c = ymin - m*xmin
  yv = m*x + c
  return yv

ymin = float(input("Ingrese el valor mínimo de la funcion: "))
ymax = float(input("Ingrese el valor máximo de la funcion: "))
xcp = my_ceropen(x,ymin=ymin,ymax=ymax)

while ymin > ymax:
  print(f"El valor mínimo {ymin} debe ser menor que el valor máximo {ymax}")
  ymin = float(input("Ingrese el valor mínimo de la funcion: "))
  ymax = float(input("Ingrese el valor máximo de la funcion: "))
  xcp = my_ceropen(x,ymin=ymin,ymax=ymax)
else:
  plt.plot(tv, xcp, 'b',label='Cero y pendiente')
  plt.grid(True)
  plt.legend()
  plt.xlabel('t')
  plt.ylabel('$X(t)$')
  plt.show()


In [ ]:
nbits = int(input("Ingrese la cantidad de bits: "))
while nbits < 1:
  print(f"La cantidad de bits debe ser mayor a 1")
  nbits = int(input("Ingrese la cantidad de bits: "))
else:
  num_est = 2**nbits
  vec = np.linspace(ymin,ymax,num_est) # vector de cauntizacion
  print(vec)

In [ ]:
from scipy.spatial.distance import cdist
def my_cuantizador(yn, vq) : #yn punto a #cuantizar, vq vector de estados

  Ne = vq.shape[0] #tamaño vector de estados
  dn = cdist(yn.reshape(-1,1),vq.reshape(-1,1))#distancia yn a vector estados, reshape(-1,1) asegura vectores columna
  ind = np.argmin(dn) #posición distancia min
  return vq[ind]

#recorrer el vector cero y spam discretizado para cuantizar
for i in range(xcp.shape[0]):
   if i==0: #primera muestra
     yq = my_cuantizador(xcp[i],vec)
   else:
     yq = np.append(yq,my_cuantizador(xcp[i],vec)) #apilar datos cuantizando si i>0

#grafica de las señales

plt.plot(tv,xcp,c='r', label='xcp')#cs continua

plt.stem(tv,xcp, label='xcp_dis')#cs discreta

#estados de cuantización

for v in vec:
  plt.plot(tv,v*np.ones(len(tv)))

# graficar señal Digital (discreta + cuantizada)

plt.plot(tv,yq,c='y',marker='o',label='ydig', linewidth=2)

#Grafica de la señal digital (Discreta + Cauntizada)

plt.plot(tv, yq, c= 'r', marker = 'o', label='Señal digital', linewidth = 2)
plt.grid(True)
plt.legend()
plt.xlabel('t')
plt.ylabel('Amplitud')
plt.show( )



# Pregunta 2 (valor 2.5 puntos)

Se dispone de un sistema modelado como una "caja negra" (ver celdas de código). Su tarea es analizar y comprobar mediante simulaciones si el sistema cumple con las propiedades de linealidad e invariancia en el tiempo. En caso de que el sistema sea lineal e invariante con el tiempo, determine su respuesta al impulso y utilice esta respuesta para calcular la salida del sistema ante la siguiente señal:

$x[n] = \sin[100 \pi n ] + \sin[600 \pi n]$

In [ ]:
# cargar sistema
FILEID = "1J9rhh0wWHZSBd8XmWGt1ZpCsMDuoUFmm"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O P1_model.zip && rm -rf /tmp/cookies.txt
!unzip -o P1_model.zip
!dir

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import joblib
from P1_model import system_
from scipy.signal import firwin, freqz, lfilter, filtfilt

#sistema pregunta 2
my_system = system_.My_System()
my_system.create_()
fs = my_system.fs #frecuencia de muestreo
t = np.arange(-0.01, 0.02, 1/fs)  # Tiempo
signal_u = np.heaviside(t,1) # función heaviside
y_u = my_system.predict(signal_u)


# Visualización de las señales
fig, axs = plt.subplots(2,1)
axs[0].stem(t, signal_u, label='Señal de entrada')
axs[0].set_xlabel('Tiempo (s)')
axs[0].set_ylabel('Amplitud')
axs[0].legend()
axs[0].grid()
axs[1].stem(t,y_u, label='Señal salida')
axs[1].set_xlabel('Tiempo (s)')
axs[1].set_ylabel('Amplitud')
axs[1].legend()
axs[1].grid()
plt.tight_layout()
plt.show()

In [ ]:
#Se verifica la linealidad
t = np.arange(-0.01, 0.02, 1/fs)  # Tiempo

a1 = 2
a2 = 3

signal_u = np.heaviside(t,1) # función heaviside
y_esca = my_system.predict(signal_u)

signal_v = np.sin(600*np.pi*t) # funcion seno
y_v = my_system.predict(signal_v)

se_1 = a1 * y_esca #señal uno miltiplicada por un escalar
se_2 = a2 * y_v #señal dos miltiplicada por un escalar


su_s = (a1 * signal_u) + (a2 * signal_v) #suma de señales
s_comb = my_system.predict(su_s)

sf = se_1 + se_2 #suma de señales escaladas

#visualizacion de las señales
fig, axs = plt.subplots(2,1)
axs[0].stem(t, s_comb, label='Señal de entrada')
axs[0].set_xlabel('Tiempo (s)')
axs[0].set_ylabel('Amplitud')
axs[0].legend()
axs[0].grid()
axs[1].stem(t,sf, label='Señal de salida')
axs[1].set_xlabel('Tiempo (s)')
axs[1].set_ylabel('Amplitud')
axs[1].legend()
axs[1].grid()
plt.tight_layout()
plt.show()



In [ ]:
#Se verifica si la señal es invariante en el tiempo

# Desplazar la señal de entrada
k= 0.005  # Número de muestras a desplazar / tiempo de inicio
x=np.heaviside(t-k,1)

# Obtener la salida del sistema para la señal desplazada
y_u_shifted = my_system.predict(x)

fig, axs = plt.subplots(2,1)
axs[0].stem(t, x, label='Señal de entrada desplazada')
axs[0].set_xlabel('Tiempo (s)')
axs[0].set_ylabel('Amplitud')
axs[0].legend
axs[0].grid()
axs[1].stem(t,y_u_shifted, label='Señal salida desplazada')
axs[1].set_xlabel('Tiempo (s)')
axs[1].set_ylabel('Amplitud')
axs[1].legend()
axs[1].grid()


In [ ]:
t_ajustado = t[:-1] #Se elimina el uiltimo elemnto de t para ajustar su tamaño y que coincida con la resouesta impulso

#Se procede a realizar la convolución
n = fs*t #Muestras
h = np.diff(signal_u) * fs # Respuesta impulso
xn = np.sin(100*np.pi*n) + np.sin(600*np.pi*n) #Señal a convolucionar con la respuesta impulso

#funcion convolucion
y_salida = np.convolve(xn,h, mode = 'same') / fs

#Visualizacion de la saldia
plt.figure()
plt.plot(n, y_salida, 'g', label = 'Salida del sistema')
plt.stem(n, y_salida, label = 'Salida del sistema')
plt.xlabel('Muestras')
plt.ylabel('Amplitud')
plt.legend()
plt.grid()
plt.title('Salida del sistema y[n]')
plt.show()